# CHAPTER 7
---
# Data Wrangling: Clean, Transform, Merge, Reshape

In [1]:
import pandas as pd
import numpy as np

## Combining and Merging Data Sets <font color='green'>[Essential]</font>
- pandas.merge connects rows in DataFrames based on one or more keys. This will
be familiar to users of SQL or other relational databases, as it implements database
join operations.
- pandas.concat glues or stacks together objects along an axis.

### Database-style DataFrame Merges <font color='green'>[Essential]</font>

In [2]:
df1 = pd.DataFrame({'key': ['b', 'b', 'a', 'c', 'a', 'a', 'b'], 'data1': range(7)})
df2 = pd.DataFrame({'key': ['a', 'b', 'd'],'data2': range(3)})

In [3]:
df1

,key,data1
0,b,0
1,b,1
2,a,2
3,c,3
4,a,4
5,a,5
6,b,6


In [4]:
df2

,key,data2
0,a,0
1,b,1
2,d,2


In [5]:
# INNER on ne garde que les lignes définies dans les deux tables
pd.merge(df1, df2, how='inner', on='key')  # on note que l'index a changé !

,key,data1,data2
0,b,0,1
1,b,1,1
2,b,6,1
3,a,2,0
4,a,4,0
5,a,5,0


In [6]:
# LEFT on ne garde que les lignes définies dans la table gauche, ici df1
pd.merge(df1, df2, how='left', on='key')

,key,data1,data2
0,b,0,1.0
1,b,1,1.0
2,a,2,0.0
3,c,3,NaN
4,a,4,0.0
5,a,5,0.0
6,b,6,1.0


In [7]:
pd.merge(df1, df2, how='right', on='key')

,key,data1,data2
0,b,0.0,1
1,b,1.0,1
2,b,6.0,1
3,a,2.0,0
4,a,4.0,0
5,a,5.0,0
6,d,NaN,2


In [8]:
# OUTER on garde toutes les valeurs
pd.merge(df1, df2, how='outer', on='key')

,key,data1,data2
0,b,0.0,1.0
1,b,1.0,1.0
2,b,6.0,1.0
3,a,2.0,0.0
4,a,4.0,0.0
5,a,5.0,0.0
6,c,3.0,NaN
7,d,NaN,2.0


In [9]:
df3 = pd.DataFrame({'lkey': ['b', 'b', 'a', 'c', 'a', 'a', 'b'], 'data1': range(7)})

In [10]:
df4 = pd.DataFrame({'rkey': ['a', 'b', 'd'],'data2': range(3)})

In [11]:
df3

,lkey,data1
0,b,0
1,b,1
2,a,2
3,c,3
4,a,4
5,a,5
6,b,6


In [12]:
df4

,rkey,data2
0,a,0
1,b,1
2,d,2


In [13]:
pd.merge(df3, df4, left_on='lkey', right_on='rkey')

,lkey,data1,rkey,data2
0,b,0,b,1
1,b,1,b,1
2,b,6,b,1
3,a,2,a,0
4,a,4,a,0
5,a,5,a,0


In [14]:
# suffixes
df5 = df4.copy()
df5['data1'] = 5

In [15]:
pd.merge(df3, df5, left_on='lkey', right_on='rkey')

,lkey,data1_x,rkey,data2,data1_y
0,b,0,b,1,5
1,b,1,b,1,5
2,b,6,b,1,5
3,a,2,a,0,5
4,a,4,a,0,5
5,a,5,a,0,5


In [16]:
pd.merge(df3, df5, left_on='lkey', right_on='rkey', suffixes=['_df3', '_df5'])

,lkey,data1_df3,rkey,data2,data1_df5
0,b,0,b,1,5
1,b,1,b,1,5
2,b,6,b,1,5
3,a,2,a,0,5
4,a,4,a,0,5
5,a,5,a,0,5


In [17]:
left = pd.DataFrame(
    {
        'key1': ['foo', 'foo', 'bar'],
        'key2': ['one', 'two', 'one'],
        'lval': [1, 2, 3]
    }
)
right = pd.DataFrame(
    {
        'key1': ['foo', 'foo', 'bar', 'bar'], 
        'key2': ['one', 'one', 'one', 'two'],
        'rval': [4, 5, 6, 7]
    }
)

In [18]:
left

,key1,key2,lval
0,foo,one,1
1,foo,two,2
2,bar,one,3


In [19]:
right

,key1,key2,rval
0,foo,one,4
1,foo,one,5
2,bar,one,6
3,bar,two,7


In [20]:
pd.merge(left, right, on=['key1', 'key2'], how='outer')

,key1,key2,lval,rval
0,foo,one,1.0,4.0
1,foo,one,1.0,5.0
2,foo,two,2.0,NaN
3,bar,one,3.0,6.0
4,bar,two,NaN,7.0


### Merging on Index <font color='#D22328'>[Advanced]</font>
C'est possible, mais on peut toujours se ramener à la syntaxe des colonnes

In [21]:
left2 = pd.DataFrame(
    [[1., 2.], [3., 4.], [5., 6.]], 
    index=['a', 'c', 'e'],
    columns=['Ohio', 'Nevada']
)
right2 = pd.DataFrame(
    [[7., 8.], [9., 10.], [11., 12.], [13, 14]],
    index=['b', 'c', 'd', 'e'], 
    columns=['Missouri', 'Alabama']
)

In [22]:
pd.merge(left2, right2, how='outer', left_index=True, right_index=True)

,Ohio,Nevada,Missouri,Alabama
a,1.0,2.0,NaN,NaN
b,NaN,NaN,7.0,8.0
c,3.0,4.0,9.0,10.0
d,NaN,NaN,11.0,12.0
e,5.0,6.0,13.0,14.0


In [23]:
left2.join(right2, how='outer') # join offre une syntaxe plus concise que merge quand on fusionne sur les index

,Ohio,Nevada,Missouri,Alabama
a,1.0,2.0,NaN,NaN
b,NaN,NaN,7.0,8.0
c,3.0,4.0,9.0,10.0
d,NaN,NaN,11.0,12.0
e,5.0,6.0,13.0,14.0


### Concatenating Along an Axis <font color='green'>[Essential]</font>

In [24]:
s1 = pd.Series([0, 1], index=['a', 'b'])
s2 = pd.Series([2, 3, 4], index=['c', 'd', 'e'])
s3 = pd.Series([5, 6], index=['f', 'g'])

pd.concat([s1, s2, s3])

a    0
b    1
c    2
d    3
e    4
f    5
g    6
dtype: int64

In [25]:
# on peut garder la trace de la série d'origine on utilisant le paramètre 'keys'
pd.concat([s1, s2, s3], keys=['one', 'two', 'three'])

one    a    0
       b    1
two    c    2
       d    3
       e    4
three  f    5
       g    6
dtype: int64

In [26]:
# en concaténant selon l'axe des colonnes, on fait apparaître des trous
pd.concat([s1, s2, s3], axis=1, keys=['one', 'two', 'three']) 

C:\Users\qchasserieau\AppData\Local\Continuum\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  


,one,two,three
a,0.0,NaN,NaN
b,1.0,NaN,NaN
c,NaN,2.0,NaN
d,NaN,3.0,NaN
e,NaN,4.0,NaN
f,NaN,NaN,5.0
g,NaN,NaN,6.0


In [27]:
# La syntaxe se généralise aux DataFrames
arr = np.arange(12).reshape((3, 4))

In [28]:
df = pd.DataFrame(arr)

In [29]:
df

,0,1,2,3
0,0,1,2,3
1,4,5,6,7
2,8,9,10,11


In [30]:
pd.concat([df, df])

,0,1,2,3
0,0,1,2,3
1,4,5,6,7
2,8,9,10,11
0,0,1,2,3
1,4,5,6,7
2,8,9,10,11


In [31]:
pd.concat([df, df], axis=1)

,0,1,2,3,0,1,2,3
0,0,1,2,3,0,1,2,3
1,4,5,6,7,4,5,6,7
2,8,9,10,11,8,9,10,11


### Combining Data with Overlap <font color='#D22328'>[Advanced]</font>
Another data combination situation can’t be expressed as either a merge or concatenation
operation. You may have two datasets whose indexes overlap in full or part.

## Reshaping and Pivoting <font color='#D22328'>[Advanced]</font>

### Reshaping with Hierarchical Indexing <font color='#D22328'>[Advanced]</font>

Hierarchical indexing provides a consistent way to rearrange data in a DataFrame.
There are two primary actions:
- stack: this “rotates” or pivots from the columns in the data to the rows
- unstack: this pivots from the rows into the columns

In [32]:
data = pd.DataFrame(
    np.arange(6).reshape((2, 3)), 
    index=pd.Index(['Ohio', 'Colorado'], name='state'),
    columns=pd.Index(['one', 'two', 'three'], name='number')
)

In [33]:
data

number,one,two,three
state,,,
Ohio,0,1,2
Colorado,3,4,5


In [34]:
result = data.stack()
result

state     number
Ohio      one       0
          two       1
          three     2
Colorado  one       3
          two       4
          three     5
dtype: int32

In [35]:
result.unstack('state')

state,Ohio,Colorado
number,,
one,0,3
two,1,4
three,2,5


In [36]:
df = pd.DataFrame(
    {'left': result, 'right': result + 5},
    columns=pd.Index(['left', 'right'], name='side')
)

In [37]:
df

side             left  right
state    number             
Ohio     one        0      5
         two        1      6
         three      2      7
Colorado one        3      8
         two        4      9
         three      5     10

In [38]:
df.unstack('state').stack('side')

state         Colorado  Ohio
number side                 
one    left          3     0
       right         8     5
two    left          4     1
       right         9     6
three  left          5     2
       right        10     7

### Pivoting “long” to “wide” Format

## Data Transformation <font color='#D22328'>[Advanced]</font>

### Removing Duplicates <font color='#D22328'>[Advanced]</font>

In [39]:
data = pd.DataFrame({'k1': ['one'] * 3 + ['two'] * 4,'k2': [1, 1, 2, 3, 3, 4, 4]})

In [40]:
data

,k1,k2
0,one,1
1,one,1
2,one,2
3,two,3
4,two,3
5,two,4
6,two,4


In [41]:
data.duplicated()

0    False
1     True
2    False
3    False
4     True
5    False
6     True
dtype: bool

In [42]:
data.drop_duplicates()

,k1,k2
0,one,1
2,one,2
3,two,3
5,two,4


In [43]:
data.drop_duplicates(subset=['k1'])

,k1,k2
0,one,1
3,two,3


### Transforming Data Using a Function or Mapping 
la méthode map offre un alternative à la syntaxe apply

### Replacing Values <font color='#D22328'>[Advanced]</font>

In [44]:
data = pd.Series([1., -999., 2., -999., -1000., 3.])

In [45]:
data

0       1.0
1    -999.0
2       2.0
3    -999.0
4   -1000.0
5       3.0
dtype: float64

In [46]:
data.replace(-999, np.nan)

0       1.0
1       NaN
2       2.0
3       NaN
4   -1000.0
5       3.0
dtype: float64

In [47]:
data.replace([-999, -1000], np.nan)

0    1.0
1    NaN
2    2.0
3    NaN
4    NaN
5    3.0
dtype: float64

In [48]:
data.replace([-999, -1000], [np.nan, 0])

0    1.0
1    NaN
2    2.0
3    NaN
4    0.0
5    3.0
dtype: float64

### Renaming Axis Indexes <font color='#D22328'>[Advanced]</font>

In [49]:
data = pd.DataFrame(
    np.arange(12).reshape((3, 4)),
    index=['Ohio', 'Colorado', 'New York'],
    columns=['one', 'two', 'three', 'four']
)

In [50]:
data

,one,two,three,four
Ohio,0,1,2,3
Colorado,4,5,6,7
New York,8,9,10,11


In [51]:
# voici un usage de la méthode map : on ne peut pas utiliser apply sur des indexes !
data.index = data.index.map(str.upper) 
data.columns = data.columns.map(str.lower)

In [52]:
data

,one,two,three,four
OHIO,0,1,2,3
COLORADO,4,5,6,7
NEW YORK,8,9,10,11


In [53]:
data.rename(index={'OHIO': 'INDIANA'}, columns={'three': 'peekaboo'})

,one,two,peekaboo,four
INDIANA,0,1,2,3
COLORADO,4,5,6,7
NEW YORK,8,9,10,11


### Discretization and Binning <font color='#D22328'>[Advanced]</font>

In [54]:
ages = [20, 22, 25, 27, 21, 23, 37, 31, 61, 45, 41, 32]
bins = [18, 25, 35, 60, 100]
group_names = ['Youth', 'YoungAdult', 'MiddleAged', 'Senior']

In [55]:
cats = pd.cut(ages, bins, labels=group_names)

In [56]:
cats

[Youth, Youth, Youth, YoungAdult, Youth, ..., YoungAdult, Senior, MiddleAged, MiddleAged, YoungAdult]
Length: 12
Categories (4, object): [Youth < YoungAdult < MiddleAged < Senior]

In [57]:
pd.value_counts(cats)

Youth         5
MiddleAged    3
YoungAdult    3
Senior        1
dtype: int64

### Detecting and Filtering Outliers <font color='#D22328'>[Advanced]</font>

In [58]:
np.random.seed(12345)
data = pd.DataFrame(np.random.randn(1000, 4))

In [59]:
data

,0,1,2,3
0,-0.204708,0.478943,-0.519439,-0.555730
1,1.965781,1.393406,0.092908,0.281746
2,0.769023,1.246435,1.007189,-1.296221
3,0.274992,0.228913,1.352917,0.886429
4,-2.001637,-0.371843,1.669025,-0.438570
5,-0.539741,0.476985,3.248944,-1.021228
6,-0.577087,0.124121,0.302614,0.523772
7,0.000940,1.343810,-0.713544,-0.831154
8,-2.370232,-1.860761,-0.860757,0.560145
9,-1.265934,0.119827,-1.063512,0.332883


In [60]:
data.describe()

,0,1,2,3
count,1000.000000,1000.000000,1000.000000,1000.000000
mean,-0.067684,0.067924,0.025598,-0.002298
std,0.998035,0.992106,1.006835,0.996794
min,-3.428254,-3.548824,-3.184377,-3.745356
25%,-0.774890,-0.591841,-0.641675,-0.644144
50%,-0.116401,0.101143,0.002073,-0.013611
75%,0.616366,0.780282,0.680391,0.654328
max,3.366626,2.653656,3.260383,3.927528


In [61]:
col = data[3]

In [62]:
col.loc[np.abs(col) > 3]

97     3.927528
305   -3.399312
400   -3.745356
Name: 3, dtype: float64

In [63]:
help((np.abs(data) > 3).any)

Help on method any in module pandas.core.frame:

any(axis=0, bool_only=None, skipna=True, level=None, **kwargs) method of pandas.core.frame.DataFrame instance
    Return whether any element is True, potentially over an axis.
    
    Returns False unless there at least one element within a series or
    along a Dataframe axis that is True or equivalent (e.g. non-zero or
    non-empty).
    
    Parameters
    ----------
    axis : {0 or 'index', 1 or 'columns', None}, default 0
        Indicate which axis or axes should be reduced.
    
        * 0 / 'index' : reduce the index, return a Series whose index is the
          original column labels.
        * 1 / 'columns' : reduce the columns, return a Series whose index is the
          original index.
        * None : reduce all axes, return a scalar.
    
    bool_only : bool, default None
        Include only boolean columns. If None, will attempt to use everything,
        then use only boolean data. Not implemented for Series.
    s

In [64]:
data[(np.abs(data) > 3).any(1)]

,0,1,2,3
5,-0.539741,0.476985,3.248944,-1.021228
97,-0.774363,0.552936,0.106061,3.927528
102,-0.655054,-0.565230,3.176873,0.959533
305,-2.315555,0.457246,-0.025907,-3.399312
324,0.050188,1.951312,3.260383,0.963301
400,0.146326,0.508391,-0.196713,-3.745356
499,-0.293333,-0.242459,-3.056990,1.918403
523,-3.428254,-0.296336,-0.439938,-0.867165
586,0.275144,1.179227,-3.184377,1.369891
808,-0.362528,-3.548824,1.553205,-2.186301


In [65]:
outliers = (np.abs(data) > 3).any(axis=1)
data.loc[outliers]

,0,1,2,3
5,-0.539741,0.476985,3.248944,-1.021228
97,-0.774363,0.552936,0.106061,3.927528
102,-0.655054,-0.565230,3.176873,0.959533
305,-2.315555,0.457246,-0.025907,-3.399312
324,0.050188,1.951312,3.260383,0.963301
400,0.146326,0.508391,-0.196713,-3.745356
499,-0.293333,-0.242459,-3.056990,1.918403
523,-3.428254,-0.296336,-0.439938,-0.867165
586,0.275144,1.179227,-3.184377,1.369891
808,-0.362528,-3.548824,1.553205,-2.186301


### Computing Indicator/Dummy Variables <font color='#D22328'>[Advanced]</font>

In [66]:
df = pd.DataFrame( {'key': ['b', 'b', 'a', 'c', 'a', 'b'],'data1': range(6)})

In [67]:
df

,key,data1
0,b,0
1,b,1
2,a,2
3,c,3
4,a,4
5,b,5


In [68]:
pd.get_dummies(df['key'])

,a,b,c
0,0,1,0
1,0,1,0
2,1,0,0
3,0,0,1
4,1,0,0
5,0,1,0


## String Manipulation <font color='#D22328'>[Advanced]</font>
On peut se servir de l'attribut str des Series pour gérer les nan

In [69]:
data = {'Dave': 'dave@google.com', 'Steve': 'steve@gmail.com','Rob': 'rob@gmail.com', 'Wes': np.nan}
data = pd.Series(data)

In [70]:
data

Dave     dave@google.com
Steve    steve@gmail.com
Rob        rob@gmail.com
Wes                  NaN
dtype: object

In [71]:
data.str.contains('gmail')

Dave     False
Steve     True
Rob       True
Wes        NaN
dtype: object